In [376]:
import copy
import numpy as np

In [377]:
class Puzzle:
    def __init__(self, board=None):
        # red = 0, green = 1, yellow = 2
        if board is None:
            # 6x6 grid with 3 random colors (0, 1, 2)
            self.board = np.random.randint(0, 3, (6, 6))
        else:
            self.board = board
        self.g = 0
        self.h = 0
        self.parent = None
        self.action_color = None
        self.start_color = self.board[0, 0]

    def printState(self):
        print("*******************")
        print(self.board)
        print("*******************")

    def floodFill(self, color):
        new_board = copy.deepcopy(self.board)
        self.floodFillUtil(new_board, 0, 0, self.start_color, color)
        return Puzzle(new_board)

    def floodFillUtil(self, board, x, y, prev_color, new_color):
        if x < 0 or x >= 6 or y < 0 or y >= 6:
            return
        if board[x][y] != prev_color or board[x][y] == new_color:
            return

        board[x][y] = new_color

        self.floodFillUtil(board, x + 1, y, prev_color, new_color)
        self.floodFillUtil(board, x - 1, y, prev_color, new_color)
        self.floodFillUtil(board, x, y + 1, prev_color, new_color)
        self.floodFillUtil(board, x, y - 1, prev_color, new_color)

    def __eq__(self, otherNode):
        if otherNode is None:
            return False
        return np.array_equal(self.board, otherNode.board)

    def __hash__(self):
        return hash(str(self.board))


In [378]:
board = Puzzle()
board.printState()

*******************
[[2 2 2 0 1 1]
 [2 0 0 0 0 0]
 [0 0 2 1 1 0]
 [2 1 0 0 1 2]
 [1 1 1 1 2 2]
 [2 1 2 0 0 2]]
*******************


In [379]:
def goalTest(node):
    return np.all(node.board == node.board[0, 0])

In [380]:
goalTest(board)

False

In [381]:
def connectedCells(board, x, y, target_color, visited):
    if x < 0 or x >= 6 or y < 0 or y >= 6:
        return 0
    if board[x][y] != target_color or visited[x][y] == 1:
        return 0

    visited[x][y] = 1
    count = 1
    count += connectedCells(board, x + 1, y, target_color, visited)
    count += connectedCells(board, x - 1, y, target_color, visited)
    count += connectedCells(board, x, y + 1, target_color, visited)
    count += connectedCells(board, x, y - 1, target_color, visited)
    return count

def calculateConnectedCells(node):
    #  no of connected cells with the same color as [0][0]
    target_color = node.board[0, 0]
    visited = np.zeros_like(node.board)
    return connectedCells(node.board, 0, 0, target_color, visited)

In [382]:
board.printState()
calculateConnectedCells(board)

*******************
[[2 2 2 0 1 1]
 [2 0 0 0 0 0]
 [0 0 2 1 1 0]
 [2 1 0 0 1 2]
 [1 1 1 1 2 2]
 [2 1 2 0 0 2]]
*******************


4

In [383]:
def manhattanDistanceHeuristic(node):
    total_distance = 0
    target_color = node.board[0, 0]
    for x in range(6):
        for y in range(6):
            if node.board[x][y] != target_color:
                total_distance += abs(x - 0) + abs(y - 0)
    return total_distance

In [384]:

def h1(node):
    # h1: no of unique colors on the board (minus 1 for the current color)
    unique_colors = len(np.unique(node.board)) - 1
    return unique_colors

In [385]:
h1(board)

2

In [386]:
def sumOfGandH(node):
    return node.g + node.h

In [387]:
sumOfGandH(board)

0

In [388]:
def a_star_search(initialNode):
    if goalTest(initialNode):
        return initialNode

    frontier = list()  # queue
    explored = set()

    initialNode.g = 0
    initialNode.h = manhattanDistanceHeuristic(initialNode) - calculateConnectedCells(initialNode)
    frontier.append(initialNode)

    while frontier:
        frontier.sort(key=sumOfGandH)
        node = frontier.pop(0)
        explored.add(node)

        # all 3 colors as actions (excluding the current start color)
        actions = [c for c in [0, 1, 2] if c != node.board[0, 0]]

        for color in actions:
            child = node.floodFill(color)
            child.g = node.g + 1
            child.h = manhattanDistanceHeuristic(initialNode) - calculateConnectedCells(initialNode)
            child.parent = node
            child.action_color = color

            if goalTest(child):
                return child

            if child not in frontier and child not in explored:
                frontier.append(child)

    return None

In [389]:
def printSolution(state):
    if state is None:
        return
    printSolution(state.parent)
   
    # red = 0, green = 1, yellow = 2
    color = state.action_color
    if color == 0:
        action = 'Action: R'
    elif color == 1:
        action = 'Action: G'
    elif color == 2:
        action = 'Action: Y'
    
    if state.action_color is not None:
        print(action)
        state.printState()

In [390]:
print("Initial:")
board.printState()

result = a_star_search(board)
print("Solution:")
printSolution(result)

Initial:
*******************
[[2 2 2 0 1 1]
 [2 0 0 0 0 0]
 [0 0 2 1 1 0]
 [2 1 0 0 1 2]
 [1 1 1 1 2 2]
 [2 1 2 0 0 2]]
*******************
Solution:
Action: R
*******************
[[0 0 0 0 1 1]
 [0 0 0 0 0 0]
 [0 0 2 1 1 0]
 [2 1 0 0 1 2]
 [1 1 1 1 2 2]
 [2 1 2 0 0 2]]
*******************
Action: G
*******************
[[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 2 1 1 1]
 [2 1 0 0 1 2]
 [1 1 1 1 2 2]
 [2 1 2 0 0 2]]
*******************
Action: R
*******************
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 2 0 0 0]
 [2 0 0 0 0 2]
 [0 0 0 0 2 2]
 [2 0 2 0 0 2]]
*******************
Action: Y
*******************
[[2 2 2 2 2 2]
 [2 2 2 2 2 2]
 [2 2 2 2 2 2]
 [2 2 2 2 2 2]
 [2 2 2 2 2 2]
 [2 2 2 2 2 2]]
*******************
